# TO DO:
# - section 2C: Address reversing-denials or affirming-denials - 

# - Pull remaining simple features (judge, county, ...) (1 night)
# - Create topic-modeling feature set (3-4 nights)
# - Create EDA visualizations (2 nights)
# - Create models (3-4 nights)
# - Convert to web app 

# Label and Feature Creation

In this notebook, I will import the single-column dataframe of appellate opinions and create columns with labels and features. 

### A. Import Libraries and DataFrame

In [143]:
import io
import re
import pandas as pd
import pickle
import operator

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [76]:
# Open the dataframe
infile = open('ProjectData/df_clean.data', 'rb')
df = pickle.load(infile)
infile.close()

In [77]:
df.reset_index(inplace=True, drop=True)

In [78]:
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3922 entries, 0 to 3921
Data columns (total 1 columns):
Opinion    3922 non-null object
dtypes: object(1)
memory usage: 30.8+ KB


(None,                                              Opinion
 0   an unpublished opinion of the north carolina ...
 1  no. coa11-246 north carolina court of appeals ...
 2  no. coa08-347 north carolina court of appeals ...
 3  michael harrison gregory and wife, vivian greg...
 4  atlantic contracting and material company, inc...)

### B. Drop Dissents, Because They are Not Relevant to the Function

#Dissents follow the opinion, when one of the three appellate judges doesn't agree; they may trigger further appeal, but they are not law, and may skew results, so they should be dropped. 

#### Exploration of opinions containing the phrase "dissenting"

In [79]:
dissents = df.loc[df['Opinion'].str.contains("dissenting")]

In [80]:
dissents.head()

,Opinion
13,"energy investors fund, l.p., inc., kvaerner as..."
22,in the court of appeals of north carolina no....
27,"cecelia l. ford, administrator of the estate o..."
35,"carolina place joint venture, plaintiff-appell..."
45,christian emerson dysart and mildred maxwell d...


Review example opinions containing the word "dissent" to see which need to be dropped:</p>
Verbiage: (Y=actual dissent N=not a dissent)<br>
Y judge horton dissenting<br>
Y judge wynn dissents. ... wynn, judge, dissenting.<br>
Y tyson, judge dissenting.<br>
Y judge tyson concurs in part, dissents in part.<br>
Y calabria, judge, dissenting. // therefore, i respectfully dissent.<br>
Y justice martin dissenting. -- Conflicts w below -- maybe eliminate first 2000 chars? <br>
Y === wynn, judge dissenting.<br>
Y - elmore, judge, dissenting.<br>
Y. martin, chief judge, concurring in part, dissenting in part. - But later<br>
Y = campbell, judge, dissenting. i respectfully dissent <br>
Y == thomas, judge, dissenting.<br>
Y judge dillon dissents in a separate opinion. - 12,600 chars into 17,000 char opi<br>
<br>
N piazza v. kirkbride tyson, j., concurring in part, dissenting in part.<br>
N souter, j., dissenting (cit<br>
N r. justice martin dissenting. justice brady dissenting. a -- too early  (913 chars in, 80,000 char opinion)<br>
N (greene, j. dissenting) - case citation<br>
N . judge elmore dissenting. -- too early, 2400 chars in to 28,000 char opinion<br>
N m. chief judge martin concurring in part and dissenting in part. - 2731 chars into 29,000 char opinion<br>
N (hudson, j., dissenting) <br>
N   1715 chars into 23,000 char opinion<br>
------ never the (j., dissenting)<br>
<br>
Pseudocode:<br>
IF ends with "judges /2 and /2 concur." (not concurs) - there isn't a dissent (one of the two would be dissenting). <br>
ELSE if after the first 3,000 chars, of the opinion:<br>
    judge /4 (dissent*)<br>
    
#### Trimming-Dissents Function Creation and Revision
Created the below function following the preceding review; ran the function and reviewed the trimmed opinions for correctness (i.e., was something more or less than a dissent trimmed from the opinion).  The function was updated to put (judge|justice) in parens and to make the verb "dissents" instead of the noun "dissent", which was pulling references to other dissents. 


In [81]:
def trim_dissents(opinion):
    global trimmed_dissents
    if re.search("(dissent)", opinion):
        if re.search("judges\W+(?:\w+\W+){0,5}?concur\b", opinion):
            return()  
        else:
            try:
                y = re.search("(?:(judge|justice)\W+(?:\w+\W+){0,6}?dissents)", opinion[3000:])
                cut_off_point = y.start()
                opinion2 = opinion[:(cut_off_point+3000)]
                trimmed_dissents += 1
                return(opinion2)
            except:
                return(opinion)
    else:
        return(opinion)

In [82]:
# Drop dissents
trimmed_dissents = 0
new_opinions = []
for i in range(len(df.Opinion)):
    x = trim_dissents(df.Opinion[i])
    new_opinions.append(x)


In [83]:
placeholder=pd.Series(new_opinions)
df["Opinion_trimmed"] = placeholder.values

In [92]:
df['Opinion'] = df['Opinion_trimmed']
df.drop(['Opinion_trimmed'], axis=1, inplace=True)

### 1. Creating a New Column for the File Numbers
This is more experimental than functional.

In [93]:
# capture file number into new column
def coa(string_):
    try:
        pat_coa_number = re.search("no.? ?coa.? ?(\d{2}-\d{1,5})",string_)
        return pat_coa_number.group(1)
    except:
        return('00-000')

In [94]:
coa(df.Opinion[3921])

'20-112'

In [95]:
coa_numbers = []
for i in range(len(df.Opinion)):
    x = coa(df.Opinion[i])
    coa_numbers.append(x)
    
placeholder2=pd.Series(coa_numbers)
df["File_Numbers"] = placeholder2.values

In [97]:
df.head(10)

,Opinion,File_Numbers
0,an unpublished opinion of the north carolina ...,19-563
1,no. coa11-246 north carolina court of appeals ...,11-246
2,no. coa08-347 north carolina court of appeals ...,08-347
3,"michael harrison gregory and wife, vivian greg...",05-885
4,"atlantic contracting and material company, inc...",02-1087
5,an unpublished opinion of the north carolina c...,13-222
6,in the court of appeals of north carolina no....,17-112
7,in the court of appeals of north carolina no....,15-862
8,no. coa11-1447 north carolina court of appeal...,11-1447
9,an unpublished opinion of the north carolina c...,13-248


### 2. Creating the Labels (Affirmed, Reversed, etc.)

The labels were created using the regex patterns below. The ultimate fuction below was created over many iterations. Initially, there were approximately 300 errors; the model was tweaked to reduce errors while maintaining reliability. Ultimately, 25 rows were dropped as errors because the cases were not beneficial to the model (i.e., they did not include a relevant summary judgment decision, the opinion on summary judgment was entwined with other components, etc.); 454 "affirmed-in-part" rows were dropped since they're not the binary outcome needed; and 408 dismissals were dropped since they don't have a usable outcome.  

#### A. Functions to Assign and Apply Labels to the DataFrame

In [98]:
def labels(string_):
    """
    This function will extract the outcome from a given string (opinion).
    Each of the 'try' statements extracts the labels with decreasing 
    degrees of confidence (first by the one-word sentence, then within
    10 words of the statement regarding concurrence, typically near the 
    end of the opinion; then clipping the last 150 chars. of the opinion 
    and looking for the associated label-words, and finally looking for 
    label-words within 5-10 words of the often-occurring phrase, 'for the
    reasons set forth above').
    """
    try:
        try:
            try:
                try:  #this level has the highest confidence of getting an accurate label, based upon review of opinions (a single-word sentence)
                    labels = re.search("\.. ?(affirmed?)\.|\.?(reversed?)\.|(affirmed in part)|\.?(dismissed)\.",string_)
                    x = labels.group(1)
                    y = labels.group(2)
                    z = labels.group(3)
                    w = labels.group(4)
                    not_none = [x,y,z]
                    a = [i for i in not_none if i != None]
                    return a[0]
                except:  # slightly less confidence; looks for outcome word within 10 words of "concur", which frequently is at the end of a unanymous opinion
                    labels = re.search("(?:concurs?\W+(?:\w+\W+){0,40}?((affirmed in part)|reversed|affirmed|dismissed|no error|vacated)|((affirmed in part)|affirmed|reversed|dismissed|no error|vacated)\W+(?:\w+\W+){0,40}?concurs?)", string_)
                    #print("Group 0:", labels.group(0), "\nGroup 1:", labels.group(1), "\nGroup 2:", labels.group(2), "\nGroup 3:", labels.group(3), "\nGroup 4:", labels.group(4))
                    x = labels2.group(1)
                    y = labels2.group(2)
                    z = labels2.group(3)
                    w = labels2.group(4)
                    not_none = [x,y,z,w]
                    a = [i for i in not_none if i != None]
                    #print("This is resulting list a:", a)
                    return a[0]
            except: #slightly less confidence; if both of the previous methods fail, this clips the last 150 chars of the opinion for any of the outcome words
                clip = string_[-150:]
    #             print(clip)
                labels3 = re.search("('affirmed in part'|reversed|affirmed|dismissed|'affirm in part'|affirm|reverse|dismiss|improvidently allowed)",clip)
                return labels3.group(0)
        except: 
            labels4 = re.search("(?:reasons set forth?\W+(?:\w+\W+){0,5}?((affirm in part)|reverse|affirm|dismiss|no error|vacated?)|((affirm in part)|affirm|reversed?|dismiss|no error|vacated?)\W+(?:\w+\W+){0,10}?reasons set forth?)", string_)
            #print("Group 0:", labels.group(0), "\nGroup 1:", labels.group(1), "\nGroup 2:", labels.group(2), "\nGroup 3:", labels.group(3), "\nGroup 4:", labels.group(4))
            x = labels4.group(1)
            y = labels4.group(2)
            z = labels4.group(3)
            w = labels4.group(4)
            not_none = [x,y,z,w]
            a = [i for i in not_none if i != None]
            #print("This is resulting list a:", a)
            return a[0]
    except:
        return('error')

In [99]:
# Test Cell 
labels(df.Opinion[2092])

'affirmed'

In [100]:
# Apply labels to the DataFrame
labels_list = []
for i in range(len(df.Opinion)):
    x = labels(df.Opinion[i])
    labels_list.append(x)
    
labels_series = pd.Series(labels_list)
df["Result"] = labels_series.values

In [101]:
df.Result.value_counts()

affirmed                 2027
reversed                  592
affirmed in part          401
dismissed                 235
error                     214
dismiss                   187
reverse                   166
affirm                     84
improvidently allowed      14
no error                    1
vacate                      1
Name: Result, dtype: int64

#### B. Combine Similar Terms and Drop Rows Irrelevant to the Outcome

In [102]:
df['Result'].replace(['reverse','affirm', 'dismiss','no error', 'vacated', 'improvidently allowed'],
                     ['reversed','affirmed','dismissed', 'affirmed', 'reversed', 'dismissed'], inplace=True)

# The model will treat 'no error' as 'affirmed' and 'vacated' as 'reversed'

In [103]:
df.Result.value_counts()

affirmed            2112
reversed             758
dismissed            436
affirmed in part     401
error                214
vacate                 1
Name: Result, dtype: int64

In [104]:
# Drop rows with 'error', 'dismissed', and 'affirmed in part' -- see section header, above
drop_list1 = df.loc[df['Result'] == 'error']
drop_list2 = df.loc[df['Result'] == 'affirmed in part']
drop_list3 = df.loc[df['Result'] == 'dismissed']

In [105]:
drop_list = list(drop_list1.index) + list(drop_list2.index) + list(drop_list3.index)

In [106]:
df.drop(drop_list, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [107]:
df.Result.value_counts()

affirmed    2112
reversed     758
vacate         1
Name: Result, dtype: int64

#### C. Ensure Labels Have the Same Effect Throughout the DataFrame

In [108]:
def substantial_right(string_):
    """ 
    I have been treating the label "affirm" as affirming the grant 
    of a summary judgment motion, and "reverse" as reversing the grant 
    of the motion, because it is much more common for the court of appeals 
    to address GRANTS of summary judgment. Much more rarely, they will 
    review a motion denying summary judgment, and the above functions will 
    have the opposite label than intended. This function will take a string, 
    review whether it contains the key words "substantial right," and then 
    analyze the language of the opinion to see if the opinion affirms or 
    reverses the DENIAL of summary judgment. If it does, it will return a
    '1', and if not, a '0'. This can be used later as a switch to flip the 
    label assigned above.
    """
    # Screen for the keword 'substantial right'
    if string_.count("substantial right") > 0:
        x = re.search("(?:(affirm|reverse?)\W+(?:\w+\W+){0,5}?denial)", string_)
        try:
            x.group(0)
            return(1)
        except:
            return(0)

In [109]:
substantial_right_list = []
for i in range(len(df.Opinion)):
    x = substantial_right(df.Opinion[i])
    substantial_right_list.append(x)

x_series = pd.Series(substantial_right_list)
df["sub_right"] = x_series.values

In [110]:
df.loc[df['sub_right'] == 1]

,Opinion,File_Numbers,Result,sub_right
342,"estate of erik dominic williams, by and throug...",10-491,affirmed,1.0
363,no. coa11-1466 north carolina court of appeals...,11-1466,reversed,1.0
450,"estate of vera hewett, et al, plaintiffs, v. c...",08-1071,reversed,1.0
615,"james l. pierson, kathy l. pierson, lincoln m....",99-1333,affirmed,1.0
752,in the supreme court of north carolina no. 484...,00-000,reversed,1.0
781,"mitchell, brewer, richardson, adams, burge & b...",09-1020,reversed,1.0
886,"nello l. teer company, inc., plaintiff, v. jon...",06-340,reversed,1.0
958,in the court of appeals of north carolina no....,16-908,reversed,1.0
978,an unpublished opinion of the north carolina c...,02-1610,affirmed,1.0
1001,"wilson myers, administrator of the estate of t...",07-285,affirmed,1.0


In [111]:
df.Opinion[1083]

" in the court of appeals of north carolina no. coa20-87 filed: 1 september 2020 mecklenburg county, no. 18 cvd 18556 unifund ccr partners, plaintiff, v. fred hoke, defendant. appeal by defendant from order entered 4 november 2019 by judge roy h. wiggins and from order entered 15 august 2019 by judge kimberly best in mecklenburg county district court. heard in the court of appeals 12 august 2020. sessoms & rogers, p.a., by andrew e. hoke, for plaintiff-appellee. erwin, bishop, capitano & moss, p.a., by fenton t. erwin, jr. and erin c. huegel, for defendant-appellant. berger, judge. on august 15, 2019, the trial court entered an order denying fred hoke’s (“defendant”) motion to dismiss, and on november 4, 2019, the trial court granted unifund ccr partners’ (“plaintiff”) motion for summary judgment. defendant appeals, arguing that plaintiff was subject to heightened pleading requirements as a “collection agency” and “debt buyer,” and that plaintiff did not adhere to those requirements. w

NOTES - CHECK RESULTS OF FUNCTION RETURNING "1"s:
index - whether sub_right fn worked as expected
------   -------------------------------<br>
338 - y - affirm denial<br>
441 - y - reverse denial<br>
603 - y - affirm denial<br>
740 - y - affirm denial<br>
769 - ~ - reverse denial<br>
873 - N - case citation<br>
944 - y but irrelevant (not msj, compel arb)<br>
964 - y - affirm denial<br>
987 - y - affirm denial<br>
1003 - Y - affirm denial<br>
1083 - y - affirm denial BUT in dissent<br>

DROP DISSENTS? 

### 3. Create Case-Type Feature By Sorting With Keywords

I created a simple sorting function which takes a dictionary of case types with associated keywords, and then it generates a popularity count of the various keywords, returning the highest-ranking case type for a given opinion. The dictionary was revised over many iterations and reviews; for instance, some words were not unique, were misleading, or needed leading/trailing spaces. 

In [24]:
# This dictionary contains types of law with associated, typically unique keywords
case_type_dict = {'premises':['premises', 'attractive nuisance', 'dangerous condition', 'slip and fall',
                            'defective condition', 'dog bite'], 
                  'products':['negligently manufactured', 'negligent manufacture', 'negligently designed',
                              'negligent design', 'manufacturing defect', 'products liability',
                              'product liability', 'ordinary use'],
                  'car_crash':['collision', 'vehicle', 'motorist'], 
                  'med_mal':['medical malpractice','health care profession', 'same or similar community', 
                             'rule 9(j)', 'rule 702(b)'], 
                  'contract':[' formation', 'recission', 'specific performance', 'incidental damages',
                              'consequential damages', 'statute of frauds', 'complex business'], 
                  'family_law':['divorce', 'custody', 'maintenance', 'child support', 'separation agreement',
                               'prenuptual', 'postnuptual', 'premarital', 'marital home'], 
                  'estates':['intestate', 'probate', 'revocable trust', 'irrevocable trust', 'testator',
                             'holographic', 'residue', 'testate', 'partition and sale', 'undivided interest'], 
                  'landlord_tenant':['lease', 'landlord', 'security deposit', ' rent ', 'chapter 42'], 
                  'construction':['building defect', 'water intrusion', 'construction defect', 'building code'], 
                  'property':['easement', 'fee simple', 'tenants in common', 'joint tenants', 'nuisance', 
                             'eminent domain', 'escheat', 'replevin', 'zoning', 'mortgage', 'foreclosure'], 
                  'unfair_deceptive':['unfair and deceptive', 'chapter 75'],
                  'defamation':['libel', 'slander', 'defamatory', 'defamation'],
                  'governmental':['sovereign immunity', 'official capacity', 'governmental immunity'],
                  'employment':['wrongful discharge', 'discrimination', 'retaliation', 'retaliatory',
                                    'retaliatory employment discrimination act', 'discriminatory'],
                  'wrongful_death':['wrongful death']}

# 'dram_shop':['dram shop'] -- removed, only 2 that were not superseded by "car crash"

In [25]:
def case_type_sorter(dict_of_keywords, string):
    """ 
    This function takes a dictionary of case types and associated 
    keywords, assigns points for the frequency of the keywords
    of a given case type, and returns the case type with the highest
    number of points, as well as a confidence measure.  The dict_of_keywords 
    should be a dictionary of case-type keys and keyword values; the string 
    should be a single string.
    """
    counter_dict = {}

    # Iterate through dictionary, counting frequency of each keyword in the string/Opinion
    for key, values in dict_of_keywords.items():
        counter_dict[key] = 0
        for value in values:
            count = string.count(value)
            existing_count = counter_dict[key]
            counter_dict[key] = count + existing_count
    
    # Get total points for all keywords
    values = counter_dict.values()
    total_count = sum(values)
    
    # Ensure that if no case_types are matched, the type returned is 'other'
    if total_count > 0:
        likely_case_type = max(counter_dict.items(), key=operator.itemgetter(1))[0]
    else:
        likely_case_type = 'other'
    
    # Return a confidence ratio (points of most likely type / all points )
    try:
        confidence = str(round((counter_dict[likely_case_type]/total_count)*100,2))+'%'
    except:
        confidence = 'n/a'
    
    return likely_case_type, confidence

In [26]:
# Copy of the above function for case-by-case review
def case_type_test_sorter(dict_of_keywords, string):
    """ 
    This is a copy of the case_type_sorter function for case-by-case review
    """
    counter_dict = {}

    # Iterate through dictionary, counting frequency of each keyword in the string/Opinion
    for key, values in dict_of_keywords.items():
        counter_dict[key] = 0
        for value in values:
            count = string.count(value)
            existing_count = counter_dict[key]
            counter_dict[key] = count + existing_count
            
    return counter_dict

In [27]:
# Apply case_type and confidence level columns to the DataFrame

case_type_list = []
case_type_confidence = []
for i in range(len(df.Opinion)):
    y,z = case_type_sorter(case_type_dict, df.Opinion[i])
    case_type_list.append(y)
    case_type_confidence.append(z)

case_type_series = pd.Series(case_type_list)
case_confidence_series = pd.Series(case_type_confidence)
df["Case_Type"] = case_type_series.values
df["Case_Type_Confidence"] = case_confidence_series.values

In [28]:
df.sample(10)

,Opinion,File_Numbers,Result,sub_right,Case_Type,Case_Type_Confidence
378,no. coa01-1439 north carolina court of appeals...,01-1439,reversed,NaN,car_crash,50.0%
2406,"tejal vyas, llc and dr. p.k. vyas, plaintiffs,...",03-1144,affirmed,NaN,family_law,100.0%
989,in the court of appeals of north carolina no....,17-618,affirmed,NaN,governmental,100.0%
659,"bobby l. raybon, and richard h. raybon, plaint...",00-1193,reversed,NaN,landlord_tenant,74.51%
803,"renee taylor stewart, plaintiff, v. charles st...",99-1482,affirmed,NaN,family_law,88.0%
409,no. coa97-1389 north carolina court of appeals...,97-1389,affirmed,NaN,unfair_deceptive,100.0%
1379,"bartlett milling company, l.p., plaintiff, v. ...",07-329,affirmed,NaN,unfair_deceptive,85.19%
2740,"william donald britt, plaintiff v. george doug...",99-792,affirmed,NaN,car_crash,100.0%
1016,"castle mcculloch, inc., plaintiff, v. donald l...",04-514,affirmed,NaN,unfair_deceptive,90.0%
142,harland dean campbell v. city of high point no...,00-000,affirmed,NaN,other,n/a


In [29]:
# Test function to review cases and diagnose issues (ultimately to update the case_type_dict)
test_cell = int(input("Index to Test:"))
print(case_type_sorter(case_type_dict, df.Opinion[test_cell]),
      '\n\n', case_type_test_sorter(case_type_dict, df.Opinion[test_cell]),
      '\n\n', df.Opinion[test_cell])

Index to Test:0
('property', '100.0%') 

 {'premises': 0, 'products': 0, 'car_crash': 0, 'med_mal': 0, 'contract': 0, 'family_law': 0, 'estates': 0, 'landlord_tenant': 0, 'construction': 0, 'property': 11, 'unfair_deceptive': 0, 'defamation': 0, 'governmental': 0, 'employment': 0, 'wrongful_death': 0} 

  an unpublished opinion of the north carolina court of appeals does not constitute controlling legal authority. citation is disfavored, but may be permitted in accordance with the provisions of rule 30(e)(3) of the north carolina rules of appellate procedure. in the court of appeals of north carolina no. coa 19-563 filed: 4 february 2020 mecklenburg county, no. 17 cvs 23389 b.v. belk, jr., plaintiff, v. vrs magnolia plaza, llc, defendant. appeal by plaintiff from judgment entered 31 december 2018 by judge karen eady-williams in mecklenburg county superior court. heard in the court of appeals 7 january 2020. james w. surane and leslie rawls, for plaintiff-appellant. martineau king, pllc

In [ ]:
df.Case_Type.value_counts()

In [112]:
len("  an unpublished opinion of the north carolina court of appeals does not constitute controlling legal authority. citation is disfavored, but may be permitted in accordance with the provisions of rule 30(e)(3) of the north carolina rules of appellate procedure. in the court of appeals of north carolina no. coa 19-563 filed: 4 february 2020 mecklenburg county, no. 17 cvs 23389 b.v. belk, jr., plaintiff, v. vrs magnolia plaza, llc, defendant. appeal by plaintiff from judgment")

477

## 4.  Extract Feature: Trial-Court Judge

## 5. Extract Feature: County

In [144]:
def extract_county(text):
    try:
        county = re.search("((\w*) county)",text[:700])
        return county.group(2)
    except:
        return(None)

In [145]:
extract_county(df.Opinion[200])

'mecklenburg'

In [146]:
county_list = []
for i in range(len(df.Opinion)):
    x = extract_county(df.Opinion[i])
    county_list.append(x)
    
placeholder3=pd.Series(county_list)
df["County"] = placeholder3.values

# Work through list below - resolve "None" and wrong counties

In [147]:
list = df.County.value_counts()
print(list)

wake            283
mecklenburg     194
guilford        130
durham           85
forsyth          84
hanover          82
buncombe         64
cumberland       49
carteret         38
orange           36
johnston         35
gaston           35
iredell          35
union            32
brunswick        31
dare             27
henderson        26
cabarrus         26
catawba          25
watauga          24
nash             23
harnett          22
onslow           22
pitt             22
davidson         21
haywood          19
craven           18
rowan            18
                 16
beaufort         16
alamance         16
randolph         16
pasquotank       15
lee              15
columbus         14
halifax          14
moore            14
wayne            14
surry            13
caldwell         13
chatham          13
cleveland        12
robeson          12
lincoln          11
currituck        11
stanly           11
duplin           11
macon            11
transylvania     11
pender           11


## 6. Topic Modeling to Extract Features

## 7. EDA Visualizations

### A. MSJ Allowed/Reversed Over Time

### B. MSJ by Case Type